In [ ]:
# Show imports
import numpy as np
import pandas as pd
import yfinance as yf
import hvplot.pandas
import plotly.graph_objs as go
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
import os
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf

In [ ]:
#pull data from yahoo finance
data =yf.download(tickers='WMT', period='max', interval = '1d', start='2007-01-01')

In [ ]:
# Show data
print(data)

In [ ]:
# build dataframe

WMT_df = pd.DataFrame(data)
WMT_df.head(10)

# change df to csv
WMT_csv = WMT_df.to_csv('/users/manu/desktop/project_2/walmart_csv', index=True)

#Show the df
WMT_df.head(10)


In [ ]:
#Show Close Only
WMT_close = WMT_df.drop(columns =['Open','High','Low','Adj Close','Volume'])
WMT_close.head(10)

In [ ]:
WMT_returns = WMT_close.pct_change().dropna()
WMT_returns

In [ ]:
# Access to the News Api
newsapi = NewsApiClient(api_key='ce3e78ab07ed4649a677e4da44c06a82')

In [ ]:
# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

In [ ]:
# Define a function to build news df
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [ ]:
# Pull all the Walmart Stock headlines data from 2008 
WMT_news_en = newsapi.get_everything(
    q=" WMT AND Walmart AND 2008",
    language="en"
)

In [ ]:
# Show the total number of news
print(WMT_news_en["totalResults"])

In [ ]:
# Create a DataFrame with the Walmart News
WMT_en_df = create_df(WMT_news_en["articles"], "en")
WMT_en_df

In [ ]:
# Create CSV from News df
WMT_en_df.to_csv('/users/manu/desktop/project_2/WMT_csv', index=False, encoding='utf-8-sig')

In [ ]:
# define  sentiment analysis  
WMT_sentiment = []
analyzer = SentimentIntensityAnalyzer()

for article in WMT_news_en['articles']:
    try:
        date = article["publishedAt"][:10]
        text = article["content"][0:198]
        sentiment = analyzer.polarity_scores(text)
        pos = sentiment['pos']
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        compound = sentiment["compound"] 
        
        WMT_sentiment.append({
            "text": text,
            "date": date,
            "positive": pos,
            "neutral": neu,
            "negative": neg,
            "compound": compound 
        })
        
    except AttributeError:
        pass

In [ ]:
# Build a SIA DF
WMT_S_df = pd.DataFrame(WMT_sentiment) 
cols = ['date', 'positive', 'neutral', 'negative', 'compound', 'text']
WMT_S_df = WMT_S_df[cols] 
WMT_S_df.describe()

In [ ]:
#Display a candle stick chart
fig = go.Figure(data=[go.Candlestick(x=WMT_df.index,
                open=WMT_df['Open'],
                high=WMT_df['High'],
                low=WMT_df['Low'],
                close=WMT_df['Close'])])

fig.show()